### Importing packages

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

### Importing dataset and making it into the correct format

In [164]:
train=pd.read_csv("train.csv")

In [165]:
train.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [166]:
data=np.array(train)

In [167]:
type(data)

numpy.ndarray

In [168]:
m,n=data.shape

In [169]:
data.shape

(42000, 785)

In [170]:
np.random.shuffle(data)

In [171]:
data_dev=data[0:2000].T

In [172]:
data_dev.shape

(785, 2000)

In [173]:
Y_dev=data_dev[0]
X_dev=data_dev[1:n]

In [174]:
Y_dev

array([1, 5, 1, ..., 7, 9, 9], dtype=int64)

In [175]:
Y_dev=Y_dev.reshape(1,2000)

In [176]:
Y_dev

array([[1, 5, 1, ..., 7, 9, 9]], dtype=int64)

In [177]:
X_dev.shape

(784, 2000)

In [178]:
n

785

In [179]:
data_train=data[2000:m].T

In [180]:
data_train.shape

(785, 40000)

In [181]:
Y_train=data_train[0].reshape(1,40000)

In [182]:
X_train=data_train[1:n]

In [183]:
X_train.shape

(784, 40000)

In [184]:
Y_train.shape

(1, 40000)

In [185]:
Y_train[0].size

40000

In [186]:
Y_train.max()

9

In [187]:
X_train.max()

255

In [188]:
X_train=X_train/255

In [189]:
X_dev=X_dev/255

In [190]:
np.arange(Y_train[0].size)

array([    0,     1,     2, ..., 39997, 39998, 39999])

In [191]:
X_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [192]:
Y_train

array([[5, 6, 3, ..., 3, 6, 6]], dtype=int64)

In [193]:
X_dev

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [194]:
Y_dev

array([[1, 5, 1, ..., 7, 9, 9]], dtype=int64)

### One hot encoding of the output

In [195]:
def one_hot(Y):
    temp_Y=np.zeros((10,Y.shape[1]))
    temp_Y[Y[0],np.arange(Y.size)]=1
    return temp_Y

In [196]:
#Y_train=one_hot(Y_train)

In [197]:
Y_train.shape

(1, 40000)

In [198]:
Y_train

array([[5, 6, 3, ..., 3, 6, 6]], dtype=int64)

In [199]:
#Y_dev=one_hot(Y_dev)

In [200]:
Y_dev

array([[1, 5, 1, ..., 7, 9, 9]], dtype=int64)

In [201]:
Y_dev.shape

(1, 2000)

In [202]:
X_train.shape

(784, 40000)

The thought process is we will first build the needed helper functions to compute the subparts of the process and then merge them into one function . Once we have built and learned the right parameters, then we can make prediction on new data.

### layer sizes , i.e no. of layers and their size

In [203]:
def layer_sizes(X,Y):
    n_x=X.shape[0]
    n_y=10
    n_h=10
    return n_x, n_h, n_y

### Initialize_parameters

In [204]:
def initialize_parameters(n_x,n_h,n_y):
    W1=np.random.randn(n_h,n_x)*0.01
    W2=np.random.randn(n_y,n_h)*0.01
    b1=np.zeros((n_h,1))
    b2=np.zeros((n_y,1))
#     W1 = np.random.rand(10, 784) - 0.5
#     b1 = np.random.rand(10, 1) - 0.5
#     W2 = np.random.rand(10, 10) - 0.5
#     b2 = np.random.rand(10, 1) - 0.5
#     parameters={
#         "W1":W1,
#         "W2":W2,
#         "b1":b1,
#         "b2":b2
#     }
    return W1, b1, W2, b2

### Defining activation function for the two layers

In [205]:
def relu(z):
    return np.maximum(z,0)

In [206]:
def softmax(Z):
    return np.exp(Z)/sum(np.exp(Z))

### Defining derivative of the activation function

In [207]:
def der_relu(Z):
    return Z>0

### Defining Forward propagation

In [208]:
def forward_propagation(X,W1,b1,W2,b2):
    Z1=np.dot(W1,X)+b1
    A1=relu(Z1)
    Z2=np.dot(W2,A1)+b2
    A2=softmax(Z2)
    return Z1,A1,Z2,A2

### Computing the cost 

In [209]:
def compute_cost(A2,Y):
    m=Y.shape[1]
    l=(Y*np.log(A2))+((1-Y)*np.log(1-A2))
    cost=(-1/m)*np.sum(l)
    cost=float(np.squeeze(cost))
    return cost

### Defining backward propagation 

In [210]:
def backward_propagation(Z1,A1,Z2,A2,W1,W2,X,Y):
    m=Y.size
    one_hot_Y=one_hot(Y)
    dZ2=A2-Y
    dW2=(1/m)*np.dot(dZ2,A1.T)
    db2=(1/m)*np.sum(dZ2,axis=1,keepdims=True)
    dZ1=(np.dot(dW2.T,dZ2))*(der_relu(Z1))
    dW1=(1/m)*np.dot(dZ1,X.T)
    db1=(1/m)*np.sum(dZ1,axis=1,keepdims=True)
#     grad={
#         "dW2":dW2,
#         "db2":db2,
#         "dW1":dW1,
#         "db1":db1
#     }
#     caches={
#     'cache':cache,
#     'parameters':parameters,
#     'cng_prm':cng_prm
#     }
    return dW2,db2,dW1,db1

## Defining update parameters function

In [211]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

## Functions for finding accuracy

In [212]:
def get_prediction(A2):
    return np.argmax(A2,0)

In [222]:
def get_accuracy(AL,Y):
    #print(prediction,Y)
    return np.sum(np.argmax(AL,0)==np.argmax(Y,0))/Y.shape[1]

In [221]:
Y_train.shape

(1, 40000)

## Creating the model

In [218]:
def nn_model(X,Y,iterations,alpha):
    n_x, n_h, n_y=layer_sizes(X,Y)
    #one_hot_Y=one_hot(Y)
    W1, b1, W2, b2=initialize_parameters(n_x,n_h,n_y)
    for i in range(iterations):
        Z1,A1,Z2,A2=forward_propagation(X,W1, b1, W2, b2)
        #cost=compute_cost(A2,one_hot_Y)
        dW2,db2,dW1,db1=backward_propagation(Z1,A1,Z2,A2,W1,W2,X,Y)
        W1, b1, W2, b2=update_params(W1, b1, W2, b2,dW1,db1,dW2,db2,alpha)
        if(i%10 == 0):
            print("Iteration: ", i)
            print(get_accuracy(A2,Y)) 
            #print(cost)
    return W1, b1, W2, b2

In [223]:
W1, b1, W2, b2=nn_model(X_train, Y_train, 500 , 0.10)

Iteration:  0
0.222975
Iteration:  10
0.0
Iteration:  20
0.0
Iteration:  30
0.0
Iteration:  40
0.0
Iteration:  50
0.0
Iteration:  60
0.0
Iteration:  70
0.0
Iteration:  80
0.0
Iteration:  90
0.0
Iteration:  100
0.0
Iteration:  110
0.0
Iteration:  120
0.0
Iteration:  130
0.0
Iteration:  140
0.0
Iteration:  150
0.0
Iteration:  160
0.0
Iteration:  170
0.0
Iteration:  180
0.0
Iteration:  190
0.0
Iteration:  200
0.0
Iteration:  210
0.0
Iteration:  220
0.0
Iteration:  230
0.0
Iteration:  240
0.0
Iteration:  250
0.0
Iteration:  260
0.0
Iteration:  270
0.0
Iteration:  280
0.0
Iteration:  290
0.0
Iteration:  300
0.0
Iteration:  310
0.0
Iteration:  320
0.0
Iteration:  330
0.0
Iteration:  340
0.0
Iteration:  350
0.0
Iteration:  360
0.0
Iteration:  370
0.0
Iteration:  380
0.0
Iteration:  390
0.0
Iteration:  400
0.0
Iteration:  410
0.0
Iteration:  420
0.0
Iteration:  430
0.0
Iteration:  440
0.0
Iteration:  450
0.0
Iteration:  460
0.0
Iteration:  470
0.0
Iteration:  480
0.0
Iteration:  490
0.0


In [80]:
temp=one_hot(Y_train)

In [81]:
temp

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [82]:
temp.shape

(10, 40000)

In [83]:
temp.size

400000

In [87]:
let=np.random.randn(10,784)*0.01

In [89]:
np.sum(param['W1']-let)

9.407560036876248